In [1]:
import numpy as np
import torch

from kornia.feature.integrated import SIFTFeature
from kornia.utils import image_to_tensor


def cv_kpt_from_laffs_responses(laffs, responses):
    kpts = []
    for i, response in enumerate(responses[0]):
        yx = laffs[0, i, :, 2]
        kp = cv.KeyPoint(yx[0].item(), yx[1].item(), response.item(), angle=0)
        kpts.append(kp)
    return kpts


def detect_and_compute(sift_feature, img, mask):
    img_t = image_to_tensor(img, False).float() / 255.0
    (lafs, responses, descs) = sift_feature(img_t, mask)
    kpts = cv_kpt_from_laffs_responses(lafs, responses)
    descs = descs[0].detach().cpu().numpy()
    return kpts, descs


def split_points(tentative_matches, kps0, kps1):
    src_pts = np.float32([kps0[m.queryIdx].pt for m in tentative_matches]).reshape(-1, 2)
    dst_pts = np.float32([kps1[m.trainIdx].pt for m in tentative_matches]).reshape(-1, 2)
    kps0 = [kps0[m.queryIdx] for m in tentative_matches]
    kps1 = [kps1[m.trainIdx] for m in tentative_matches]
    return src_pts, dst_pts, kps0, kps1


def get_tentatives(kpts0, desc0, kpts1, desc1, ratio_threshold):
    matcher = cv.BFMatcher(crossCheck=False)
    knn_matches = matcher.knnMatch(desc0, desc1, k=2)
    matches2 = matcher.match(desc1, desc0)

    tentative_matches = []
    for m, n in knn_matches:
        if matches2[m.trainIdx].trainIdx != m.queryIdx:
            continue

        if m.distance < ratio_threshold * n.distance:
            tentative_matches.append(m)

    src, dst, kpts0, kpts1 = split_points(tentative_matches, kpts0, kpts1)
    return src, dst, kpts0, kpts1, tentative_matches


def get_visible_part_mean_absolute_reprojection_error_np(img1, img2, H_gt, H):
    """We reproject the image 1 mask to image2 and back to get the visible part mask.
    Then we average the reprojection absolute error over that area
    """

    h, w = img1.shape[:2]
    mask1 = np.ones((h, w))
    mask1in2 = cv.warpPerspective(mask1, H_gt, img2.shape[:2][::-1])
    mask1inback = cv.warpPerspective(mask1in2, np.linalg.inv(H_gt), img1.shape[:2][::-1]) > 0
    xi = np.arange(w)
    yi = np.arange(h)
    xg, yg = np.meshgrid(xi, yi)

    coords = np.concatenate([xg.reshape(*xg.shape, 1), yg.reshape(*yg.shape, 1)], axis=-1)

    xy_rep_gt = cv.perspectiveTransform(coords.reshape(-1, 1, 2).astype(float), H_gt.astype(np.float32)).squeeze(1)
    xy_rep_estimated = cv.perspectiveTransform(
        coords.reshape(-1, 1, 2).astype(np.float32), H.astype(np.float32)
    ).squeeze(1)

    error = np.sqrt(((xy_rep_gt - xy_rep_estimated) ** 2).sum(axis=1)).reshape(xg.shape) * mask1inback
    mean_error = error.sum() / mask1inback.sum()
    return mean_error


def homography_est_cv(Hs_gt, imgs):
    
    print("visible part mean absolute reprojection error using np/cp:")

    sift_feature = SIFTFeature(device=torch.device("cpu"))

    ratio_threshold = 0.8
    ransac_th = 0.5
    ransac_conf = 0.9999
    ransac_iters = 100000

    kpts_0, desc_0 = detect_and_compute(sift_feature, imgs[0], mask=None)

    for other_i in range(1, len(imgs)):

        kpts_other, desc_other = detect_and_compute(sift_feature, imgs[other_i], mask=None)
        src_pts, dst_pts, _, _, tentative_matches = get_tentatives(
            kpts_0, desc_0, kpts_other, desc_other, ratio_threshold
        )
        H_est, inlier_mask = cv.findHomography(
            src_pts, dst_pts, cv.RANSAC, maxIters=ransac_iters, ransacReprojThreshold=ransac_th, confidence=ransac_conf
        )

        H_gt = Hs_gt[other_i - 1]
        MAE = get_visible_part_mean_absolute_reprojection_error_np(imgs[0], imgs[other_i], H_gt, H_est)
        print(f"rotation by {90 * other_i} degrees: {MAE}")  
        # this can be asserted after https://github.com/kornia/kornia/pull/2105 is released
        # assert MAE < 0.01


In [2]:
import numpy as np
import torch

from kornia.feature import match_mnn
from kornia.feature.integrated import SIFTFeature
from kornia.geometry import RANSAC
from kornia.geometry.transform import warp_perspective
from kornia.utils import image_to_tensor


def get_visible_part_mean_absolute_reprojection_error_torch(img1_t, img2_t, H_gt_t, H_t, device):
    """We reproject the image 1 mask to image2 and back to get the visible part mask.
    Then we average the reprojection absolute error over that area
    """
    h, w = img1_t.shape[2:]
    mask1_t = torch.ones((1, 1, h, w), device=device)

    H_gt_t = H_gt_t[None]
    mask1in2_t = warp_perspective(mask1_t, H_gt_t, img2_t.shape[2:][::-1])
    mask1inback_t = warp_perspective(mask1in2_t, torch.linalg.inv(H_gt_t), img1_t.shape[2:][::-1]) > 0

    xi_t = torch.arange(w, device=device)
    yi_t = torch.arange(h, device=device)
    xg_t, yg_t = torch.meshgrid(xi_t, yi_t, indexing='xy')

    coords_t = torch.cat(
        [xg_t.reshape(*xg_t.shape, 1), yg_t.reshape(*yg_t.shape, 1), torch.ones(*yg_t.shape, 1, device=device)], dim=2
    )

    def get_xy_rep(H_loc):
        xy_rep_t = H_loc.to(torch.float32) @ coords_t.reshape(-1, 3, 1).to(torch.float32)
        xy_rep_t /= xy_rep_t[:, 2:3]
        xy_rep_t = xy_rep_t[:, :2]
        return xy_rep_t

    xy_rep_gt_t = get_xy_rep(H_gt_t)
    xy_rep_est_t = get_xy_rep(H_t)
    error_t = torch.sqrt(((xy_rep_gt_t - xy_rep_est_t) ** 2).sum(axis=1)).reshape(xg_t.shape) * mask1inback_t[0, 0].T
    mean_error_t = error_t.sum() / mask1inback_t.sum()

    return mean_error_t.detach().cpu().item()


def homography_est_torch(Hs_gt_t, imgs_t, device):

    print("visible part mean absolute reprojection error using torch/kornia:")
    
    ransac = RANSAC(
        model_type='homography', inl_th=0.5, batch_size=2048, max_iter=100000, confidence=0.9999, max_lo_iters=5
    )

    sf = SIFTFeature(device=device)

    lafs0, responses0, descs_t0 = sf(imgs_t[0], mask=None)
    for other_i in range(1, len(imgs_t)):

        lafs1, responses1, descs_t1 = sf(imgs_t[other_i], mask=None)
        _, matches = match_mnn(descs_t0[0], descs_t1[0], dm=None)
        src_pts_t = lafs0[0, matches[:, 0], :, 2]
        dst_pts_t = lafs1[0, matches[:, 1], :, 2]

        H_est_t, inliers = ransac(src_pts_t, dst_pts_t)

        H_gt_t = Hs_gt_t[other_i - 1]
        MAE = get_visible_part_mean_absolute_reprojection_error_torch(
            imgs_t[0], imgs_t[other_i], H_gt_t, H_est_t, device
        )
        print(f"rotation by {90 * other_i} degrees: {MAE}")  
        # this can be asserted after https://github.com/kornia/kornia/pull/2105 is released
        # assert MAE < 0.2

In [3]:
from PIL import Image, ImageOps

device = torch.device("cpu")

# this is the first img of 'boat' from https://www.robots.ox.ac.uk/~vgg/data/affine/, 
# but it could be anything, really
pil = Image.open("data/boat/img1.pgm")
img = np.array(ImageOps.grayscale(pil))

img_t = (image_to_tensor(img, False).float() / 255.0).to(device=device)
Hs_gt_t = torch.tensor(
    [
        [[0.0, 1.0, 0.0], [-1.0, 0.0, img_t.shape[-1] - 1], [0.0, 0.0, 1.0]],
        [[-1.0, 0.0, img_t.shape[-1] - 1], [0.0, -1.0, img_t.shape[-2] - 1], [0.0, 0.0, 1.0]],
        [[0.0, -1.0, img_t.shape[-2] - 1], [1.0, 0.0, 0.0], [0.0, 0.0, 1.0]],
    ],
    device=device,
)
imgs_t = [img_t] + [torch.clone(torch.rot90(img_t, i, [2, 3])) for i in range(1, 4)]
homography_est_torch(Hs_gt_t, imgs_t, device)


visible part mean absolute reprojection error using torch/kornia:
rotation by 90 degrees: 0.4954281747341156
rotation by 180 degrees: 0.7103663086891174
rotation by 270 degrees: 0.4993991255760193


In [4]:
## OpenCV (i.e. OpenCV's RANSAC) would give more stable results 

# import cv2 as cv

# Hs_gt_rot = [h.numpy() for h in Hs_gt_t]
# imgs_rot = [img] + [np.rot90(img, i, [0, 1]).copy() for i in range(1, 4)]
# homography_est_cv(Hs_gt_rot, imgs_rot)
